# MCP

In [1]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

Let's set up our RAG database connection:

In [2]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [4]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Integrate EXA Search as an MCP:

In [14]:
# Exa Search MCP code comes here:
from agents import WebSearchTool

# exa_search_mcp = MCPServerStreamableHttp(
#     name="Exa Search MCP",
#     params={
#         "url": f"https://mcp.exa.ai/mcp?exaApiKey={os.environ.get("EXA_API_KEY")}",
#         "timeout": 30,
#     },
#     client_session_timeout_seconds=30,
#     cache_tools_list=True,
#     max_retry_attempts=1,
# )

# await exa_search_mcp.connect()
web_search = WebSearchTool()
calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You MUST follow this workflow STRICTLY:
        1) ALWAYS start by using the web_search tool to find the complete recipe and all ingredients for the requested meal. This is MANDATORY even if you think you know the ingredients.
        2) Once you have the complete ingredient list from web search, use the calorie_lookup_tool for EACH individual ingredient.
        3) Never use calorie_lookup_tool for the meal name directly - always break it down into ingredients first using web_search.
    * CRITICAL: For any meal or dish (like "english breakfast", "pizza", "burger"), you MUST use web_search FIRST before any calorie lookup.
    * In your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool, web_search]
)

Reference query - shouldn't use ExaSearch:

In [8]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    - Banana: 1 medium (~118 g) ≈ 105 kcal (89 kcal per 100 g)
    - Apple: 1 medium (~182 g) ≈ 95 kcal (52 kcal per 100 g)
    - Total for one banana + one apple ≈ 200 kcal
    
    Calories per 100 g:
    - Banana: 89 kcal/100 g
    - Apple: 52 kcal/100 g
    
    Note: actual calories vary slightly with size.
- 7 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [15]:
with trace("Nutrition Assistant with MCP "):
    result = await Runner.run(
        calorie_agent_with_search, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A typical full English breakfast is about 618 kcal per serving (recipe from BBC Good Food). If you’d like, tell me your exact ingredients and quantities (e.g., 2 rashers bacon, 2 sausages, 2 eggs, beans, tomatoes, mushrooms, 2 slices toast, 1 tsp oil) and I’ll calculate a precise per-ingredient breakdown.
